# O PROBLEMA

Imagine agora que você foi contratado(a) como Expert em Data Analytics por um grande hospital para entender como foi o comportamento da população na época da pandemia da COVID-19 e quais indicadores seriam importantes para o planejamento, caso haja um novo surto da doença.

Apesar de ser contratado(a) agora, a sua área observou que a utilização do estudo do PNAD-COVID 19 do IBGE seria uma ótima base para termos boas respostas ao problema proposto, pois são dados confiáveis. Porém, não será necessário utilizar todas as perguntas realizadas na pesquisa para enxergar todas as oportunidades ali postas.

É sempre bom ressaltar que há dados triviais que precisam estar no projeto, pois auxiliam muito na análise dos dados:

## PNAD-COVID-19 do IBGE

O Head de Dados pediu para que você entrasse na base de dados do PNAD-COVID-19 do IBGE e organizasse esta base para análise, utilizando Banco de Dados em Nuvem e trazendo as seguintes características:

- a. Utilização de no máximo 20 questionamentos realizados na pesquisa;
- b. Utilizar 3 meses para construção da solução;
- c. Caracterização dos sintomas clínicos da população;
- d. Comportamento da população na época da COVID-19;
- e. Características econômicas da sociedade;

Seu objetivo será trazer uma breve análise dessas informações, como foi a organização do banco, as perguntas selecionadas para a resposta do problema e quais seriam as principais ações que o hospital deverá tomar em caso de um novo surto de COVID-19.

- Dica: Leiam com atenção a base de dados e toda a documentação que o site o PNAD-COVID 19 traz, principalmente os dicionários, que ajudam e muito no entendimento da Base de Dados.
- Dica 2: Utilizem o que já foi ensinado e consolidado nas outras fases para apresentar a resolução do projeto.

Lembre-se de que você poderá apresentar o desenvolvimento do seu projeto durante as lives com docentes. Essa é uma boa oportunidade para discutir sobre as dificuldades encontradas e pegar dicas valiosas com especialistas e colegas de turma.

Link para a base: https://www.ibge.gov.br/estatisticas/investigacoes-experimentais/estatisticas-experimentais/27946-divulgacao-semanal-pnadcovid1?t=downloads&utm_source=covid19&utm_medium=hotsite&utm_campaign=covid_19

In [28]:
import warnings 
warnings.filterwarnings(action = 'ignore')

In [29]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from pyspark.sql import SparkSession
import findspark
import pandas as pd
import plotly.express as px

### Conectando ao MySQL

In [30]:
'''def mysql_connection(host, user, passwd, database=None):
    engine = create_engine(f'mysql+pymysql://{user}:{passwd}@{host}/{database}')
    return engine.connect()

mysql = mysql_connection('127.0.0.1', 'root', 'admin', 'pnad_covid')'''

"def mysql_connection(host, user, passwd, database=None):\n    engine = create_engine(f'mysql+pymysql://{user}:{passwd}@{host}/{database}')\n    return engine.connect()\n\nmysql = mysql_connection('127.0.0.1', 'root', 'admin', 'pnad_covid')"

### Criando sessão Spark

In [31]:
findspark.init()

In [32]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [33]:
spark

### Importanto dados tratados da view do arquivo CSV

In [34]:
df = spark.read.csv('dados/dados_exportados/dados_uteis/dados_uteis_gz/2023-10-31_pnad_covid_view.csv.gz', sep=',', inferSchema=True, header=True)
df.show(5, truncate=False)

+----------+--------------+------------------+-----+------+--------+------------------------------------+-----------------------------+-----------------------+-----------------------------+---------------------------------+--------------------+------------------------+------------------+---------------------------------+--------------------------+---------------------------+-------------------------------+----------------+-------------------------------+------------------------+---------------------+-----------------------+-----------------+-----------------------+--------------------+-------------+-----------------------+-----------+---------------------------+-------------+-----------------+---------------+-----+--------+
|data      |uf            |situacao_domicilio|idade|sexo  |cor_raca|escolaridade                        |questao_estabelecimento_saude|questao_permaneceu_casa|questao_remedio_conta_propria|questao_remedio_orientacao_medica|questao_hospital_SUS|questao_hospital_priva

### Análise exploratória de características clínicas

In [35]:
df_temp = df.createOrReplaceTempView('df_temp') #criando tabela temporária na sessão Spark SQL

#### **<p style='color:gray'> Estatísticas descritivas de pacientes sintomáticos e assintomáticos</p>**

**Qual é a porcetagem de pacientes sintomáticos e assintomáticos?**

In [61]:
df_testes_positivos_sintomas_covid = spark.sql(
    '''
        SELECT 
            CASE
            WHEN sintoma_covid = 'Sim' THEN 'Sintomáticos'
            ELSE 'Assintomáticos' 
            END AS sintoma_covid,
            count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp
        WHERE resultado_teste = 'Positivo'
        GROUP BY sintoma_covid
    '''
).toPandas()

fig = px.pie(
    data_frame=df_testes_positivos_sintomas_covid.sort_values('qtd_testes_positivos', ascending=False),
    values = 'qtd_testes_positivos',
    names = 'sintoma_covid',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_positivos':'Quantidade',
        'sintoma_covid':'Condição'
    }
)

fig.update_traces(
    pull=[0.1, 0]

)

fig.update_layout(
    title='<b>Porcentagem de infectados sintomáticos e assintomáticos</b>',
    legend_title='Legenda',
    width=800,
    height=600
)

fig.show()

**Qual é porcentagem de infectados por fator de risco?**

In [66]:
df_testes_positivos_fator_risco = spark.sql(
'''
    SELECT
        (sum(CASE WHEN descricao_fator_risco_covid LIKE '%Diabetes%' THEN 1 ELSE 0 END) /sum(CASE WHEN fator_risco_covid = 'Sim' THEN 1 ELSE 0 END)) * 100 AS Diabetes,
        (sum(CASE WHEN descricao_fator_risco_covid LIKE '%Hipertensao%' THEN 1 ELSE 0 END) / sum(CASE WHEN fator_risco_covid = 'Sim' THEN 1 ELSE 0 END)) * 100  AS Hipertensao,
        (sum(CASE WHEN descricao_fator_risco_covid LIKE '%Doenca respiratoria%' THEN 1 else 0 END) / sum(CASE WHEN fator_risco_covid = 'Sim' THEN 1 ELSE 0 END)) * 100 as Doenca_respiratoria,
        (sum(CASE WHEN descricao_fator_risco_covid LIKE '%Idoso%' THEN 1 ELSE 0 END) / sum(CASE WHEN fator_risco_covid = 'Sim' THEN 1 ELSE 0 END)) * 100  AS Idoso,
        (sum(CASE WHEN descricao_fator_risco_covid LIKE '%Cancer%' THEN 1 ELSE 0 END) / sum(CASE WHEN fator_risco_covid = 'Sim' THEN 1 ELSE 0 END)) * 100  as Cancer,
        (sum(CASE WHEN descricao_fator_risco_covid LIKE '%Doenca cardiaca%' THEN 1 ELSE 0 END) / sum(CASE WHEN fator_risco_covid = 'Sim' THEN 1 ELSE 0 END)) * 100  AS Doenca_cardiaca
    FROM df_temp
    WHERE resultado_teste = 'Positivo'
'''
).toPandas()

df_testes_positivos_fator_risco = tipo_risco.T.reset_index().rename(columns={0:'percentual_testes_positivos','index': 'fator_risco_covid'})
df_testes_positivos_fator_risco.sort_values('percentual_testes_positivos', ascending=False, inplace=True)

fig = px.bar(
    data_frame = df_testes_positivos_fator_risco,
    x = 'fator_risco_covid',
    y = 'percentual_testes_positivos',
    color= 'fator_risco_covid',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'percentual_testes_positivos': 'Percentual de testes positivos',
        'fator_risco_covid': 'Fato de risco'
    }
)

fig.update_layout(
    title='<b>Porcentagem de infectados por fator de risco</b>',
    showlegend=False,
    yaxis_title='%',
    width=800,
    height=600
)

fig.show()

**Qual é porcentagem de infectados por fator de risco?**

In [ ]:
tipo_sintoma = spark.sql(
    '''
    select
          (sum(case when descricao_sintoma_covid like '%Febre%' then 1 ELSE 0 END) / sum(case when descricao_sintoma_covid is not null then 1 ELSE 0 END)) * 100 as Febre,
          sum(case when descricao_sintoma_covid like '%Tosse%'then 1 ELSE 0 END) / sum(case when descricao_sintoma_covid is not null then 1 ELSE 0 END) * 100 AS tosse,
          sum(case when descricao_sintoma_covid like '%Dor de garganta%' then 1 ELSE 0 END) / sum(case when descricao_sintoma_covid is not null then 1 ELSE 0 END) * 100 AS dor_garganta,
          sum(case when descricao_sintoma_covid like '%Dificuldade para respirar%' then 1 ELSE 0 END) / sum(case when descricao_sintoma_covid is not null then 1 ELSE 0 END) * 100 AS dificuldade_respirar,
          sum(case when descricao_sintoma_covid like '%Dor de cabeça%' then 1 ELSE 0 END) / sum(case when descricao_sintoma_covid is not null then 1 ELSE 0 END) * 100 AS dor_cabeca,
          sum(case when descricao_sintoma_covid like '%Dor no peito%'then 1 ELSE 0 END) / sum(case when descricao_sintoma_covid is not null then 1 ELSE 0 END) * 100 AS dor_peito,
          sum(case when descricao_sintoma_covid like '%Nausea%' then 1 ELSE 0 END) / sum(case when descricao_sintoma_covid is not null then 1 ELSE 0 END) * 100 AS nausea,
          sum(case when descricao_sintoma_covid like '%Fadiga%'then 1 ELSE 0 END) / sum(case when descricao_sintoma_covid is not null then 1 ELSE 0 END) * 100 AS fadiga,
          sum(case when descricao_sintoma_covid like '%Dor nos olhos%' then 1 ELSE 0 END) / sum(case when descricao_sintoma_covid is not null then 1 ELSE 0 END) * 100 AS dor_olhos,
          sum(case when descricao_sintoma_covid like '%Perda de olfato ou paladar%' then 1 ELSE 0 END) / sum(case when descricao_sintoma_covid is not null then 1 ELSE 0 END) * 100 AS perda_olfato_paladar,
          sum(case when descricao_sintoma_covid like '%Dor muscular%' then 1 ELSE 0 END) / sum(case when descricao_sintoma_covid is not null then 1 ELSE 0 END) * 100 AS dor_muscular,
          sum(case when descricao_sintoma_covid like '%Diarreia%' then 1 ELSE 0 END) / sum(case when descricao_sintoma_covid is not null then 1 ELSE 0 END) * 100 AS diarreia
    from df_covid
    WHERE resultado_teste = 'Positivo'
    '''
).toPandas()

df_tipo_sintoma = tipo_sintoma.T.reset_index().rename(columns={0:'perc_sintoma','index':'sintoma'})
df_tipo_sintoma.sort_values('perc_sintoma',ascending=False, inplace=True)

fig = px.bar(
    data_frame = df_tipo_sintoma,
    x = df_tipo_sintoma.sintoma,
    y = df_tipo_sintoma.perc_sintoma,
    color= df_tipo_sintoma.sintoma,
    color_discrete_sequence=['#8B0000']
    )

fig.update_layout(
    title='<b>Percentual de Sintomas em casos Positivos</b>',
    xaxis_title='Tipo de Sintomas',
    yaxis_title='Percentual',
    showlegend=False,
    width=1200,
    height=800
)

fig.show()

In [ ]:
internada_atendimento = spark.sql(

'''
    select
      count(resultado_teste) as qtd_internacao,
      (case
          when questao_internacao = "Sim" then 'Internado(a)'
          when questao_internacao = 'Não' then 'Não ficou Internado(a)'
            else 'Ignorado ou não respondeu' end) as questao_internacao
    from df_covid
    WHERE resultado_teste = 'Positivo'
    group by 2
    order by 1 desc
'''
).toPandas()

fig = px.bar(
    data_frame = internada_atendimento,
    x = internada_atendimento.questao_internacao,
    y = internada_atendimento.qtd_internacao,
    color= internada_atendimento.questao_internacao,
    color_discrete_sequence=['#8B0000']
    )

fig.update_layout(
    title='<b>Pessoas que ficaram internadas</b>',
    xaxis_title='Quantidade',
    yaxis_title='Pessoas internadas',
    showlegend=False,
    width=1200,
    height=800
)

fig.show()